In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

# Folder containing images
image_folder = "/content/drive/My Drive/Bubble/image/"  # Update with your path

# Define size threshold
min_bubble_area = 300  # Adjust this based on image resolution
max_bubble_area = 1000000  # Avoid detecting large unwanted regions

# Initialize results list
results = []

# Get list of all JPEG images
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg'))]

for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error loading {image_name}")
        continue

    # Convert to grayscale, apply Gaussian blur and threshold
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (11, 5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # Morphological transformation
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    # Find contours
    cnts, _ = cv2.findContours(opening, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    bubble_count = 0
    areas = []
    circularities = []

    for c in cnts:
        area = cv2.contourArea(c)
        perimeter = cv2.arcLength(c, True)

        if perimeter == 0 or area < min_bubble_area or area > max_bubble_area:
            continue  # Skip small or large non-bubble objects

        circularity = 4 * np.pi * (area / (perimeter ** 2))
        x, y, w, h = cv2.boundingRect(c)
        aspect_ratio = w / float(h)

        # Filter bubbles based on circularity, aspect ratio, and area
        if 0.75 < circularity <= 1.1 and 0.85 <= aspect_ratio <= 1.15:
            bubble_count += 1
            areas.append(area)
            circularities.append(circularity)

    # Compute statistics
    median_area = np.median(areas) if areas else 0
    avg_area = np.mean(areas) if areas else 0
    min_area = min(areas) if areas else 0
    max_area = max(areas) if areas else 0
    range_area = max_area - min_area if areas else 0
    std_dev_area = np.std(areas) if areas else 0
    avg_circularity = np.mean(circularities) if circularities else 0

    results.append({
        "Image Name": image_name,
        "Total Bubbles": bubble_count,
        "Median Area": median_area,
        "Avg Area": avg_area,
        "Min Area": min_area,
        "Max Area": max_area,
        "Area Range": range_area,
        "Std Dev Area": std_dev_area,
        "Avg Circularity": avg_circularity
    })

# Convert to DataFrame and save results
df_results = pd.DataFrame(results)
df_results.to_csv("/content/drive/My Drive/Bubble/image/bubble_statistics.csv", index=False)

# Print results
print(df_results)


                                      Image Name  Total Bubbles  Median Area  \
0       Foto el 15-07-24 a la(s) 4.01 p. m..jpeg             24      8554.00   
1       Foto el 23-07-24 a la(s) 2.39 p. m..jpeg             16       698.00   
2    Foto el 23-07-24 a la(s) 2.39 p. m. #2.jpeg             12       590.75   
3       Foto el 23-07-24 a la(s) 2.41 p. m..jpeg              3      1205.00   
4    Foto el 23-07-24 a la(s) 2.41 p. m. #2.jpeg             11       778.00   
..                                           ...            ...          ...   
228     Foto el 20-02-25 a la(s) 7.14 p. m..jpeg            168      2191.00   
229  Foto el 20-02-25 a la(s) 7.14 p. m. #2.jpeg            129      3160.00   
230  Foto el 20-02-25 a la(s) 7.14 p. m. #3.jpeg            165      1682.50   
231  Foto el 20-02-25 a la(s) 7.14 p. m. #4.jpeg            114      4167.25   
232  Foto el 20-02-25 a la(s) 7.14 p. m. #5.jpeg             77      3891.50   

         Avg Area  Min Area  Max Area  